# National Park Vacation Route Tool

## Data Source

Data was obtained from the National Park Service (https://www.nps.gov/subjects/developer/api-documentation.htm).

## Data Preparation

### Import Libraries

In [55]:
# Import requests for JSON
import requests

# Import libraries required for this analysis
import numpy as np
import pandas as pd

# For displaying plots and maps
import seaborn as sns
import matplotlib.pyplot as plt
from folium import Map, Figure

# For timing training/prediction
import time

# Display all fields
pd.set_option('display.max_columns', None)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Import Raw Data from NPS API (external script)

In [42]:
# (Moved to fetch_nps_data.py)
"""
# Available fields for each park

id
url
*fullName
parkCode
name
*description
*designation
*latitude
*longitude
latLong (combined "lat:..., long:...")
*activities (array of objects with id and name)
topics
*states
contacts (e.g., phone, email)
entranceFees (array)
entrancePasses
fees
directionsInfo
directionsUrl
operatingHours
addresses
images (photos array)
weatherInfo


API_KEY = "fpyJ9NycrgZX5mK8f0n90c4qXGPcYAsBPwt4BLJk"
url = "https://developer.nps.gov/api/v1/parks"

def fetch_all_parks(api_key):
    all_parks = []
    start = 0
    limit = 50

    while True:
        params = {
            "limit": limit,
            "start": start,
            "api_key": api_key
        }

        response = requests.get(url, params=params)
        response.raise_for_status()  # raise error for bad response
        data = response.json().get("data", [])

        if not data:
            break  # no more data

        all_parks.extend(data)
        start += limit  # go to next page

    return all_parks

# Fetch data
parks_raw = fetch_all_parks(API_KEY)

# Convert to DataFrame
records = []
for park in parks_raw:
    activity_list = park.get('activities', [])
    activity_names = [a.get('name', '') for a in activity_list]
    
    records.append({
        'name': park.get('fullName', ''),
        'latitude': park.get('latitude', ''),
        'longitude': park.get('longitude', ''),
        'designation': park.get('designation', ''),
        'states': park.get('states', ''),
        'description': park.get('description', ''),
        'activities': ', '.join(activity_names)
    })

parks = pd.DataFrame(records)

# Save as CSV
parks.to_csv("../data/nps_parks_with_activities.csv", index=False)
"""
# Run from fetch_nps_data.py
%run ../scripts/fetch_nps_data.py

### Import Generated CSV to Dataframe

In [43]:
parks = pd.read_csv('../data/nps_parks_with_activities.csv')

### Inspect Data

In [44]:
# Print dataframe head
print('First five rows of dataframe')
display(parks.head())
print()
    
# Print dataframe sample
print('Random five rows of dataframe')
display(parks.sample(5))
print()
    
# Check for missing values
print('Check for Missing Values')
print(parks.isna().sum())
print()

# Check data types
print('Check Data Types')
print(parks.info())
print()

# Check values for each column
print('Describe Dataframe')
print(parks.describe(include = 'all'))
print()
    
# Check for duplicates
print('Count of Duplicated Rows')
print(parks.duplicated().sum())

First five rows of dataframe


,name,latitude,longitude,designation,states,description,amenities,activities
0,Abraham Lincoln Birthplace National Historical...,37.585866,-85.673305,National Historical Park,KY,For over a century people from around the worl...,NaN,"Astronomy, Stargazing, Food, Picnicking, Guide..."
1,Acadia National Park,44.409286,-68.247501,National Park,ME,Acadia National Park protects the natural beau...,NaN,"Arts and Culture, Cultural Demonstrations, Ast..."
2,Adams National Historical Park,42.255396,-71.011604,National Historical Park,MA,From the sweet little farm at the foot of Penn...,NaN,"Guided Tours, Self-Guided Tours - Walking, Liv..."
3,African American Civil War Memorial,38.916600,-77.026000,NaN,DC,"Over 200,000 African-American soldiers and sai...",NaN,"Guided Tours, Self-Guided Tours - Walking"
4,African Burial Ground National Monument,40.714527,-74.004474,National Monument,NY,The African Burial Ground is the oldest and la...,NaN,"Arts and Culture, Guided Tours, Junior Ranger ..."



Random five rows of dataframe


,name,latitude,longitude,designation,states,description,amenities,activities
119,Devils Postpile National Monument,37.615256,-119.087390,National Monument,CA,Established in 1911 by presidential proclamati...,NaN,"Astronomy, Stargazing, Biking, Road Biking, Ca..."
291,Medgar and Myrlie Evers Home National Monument,32.340963,-90.212646,National Monument,MS,Medgar and Myrlie Evers were partners in the c...,NaN,NaN
394,Santa Fe National Historic Trail,37.727652,-99.921969,National Historic Trail,"CO,KS,MO,NM,OK","You can almost hear the whoops and cries of ""A...",NaN,"Arts and Culture, Theater, Auto and ATV, Sceni..."
187,Glacier National Park,48.684147,-113.800931,National Park,MT,"A showcase of melting glaciers, alpine meadows...",NaN,"Arts and Culture, Cultural Demonstrations, Liv..."
429,Trail Of Tears National Historic Trail,36.053002,-89.676104,National Historic Trail,"AL,AR,GA,IL,KY,MO,NC,OK,TN",Remember and commemorate the survival of the C...,NaN,"Arts and Culture, Craft Demonstrations, Auto a..."



Check for Missing Values
name             0
latitude         1
longitude        1
designation     35
states           0
description      0
amenities      474
activities      10
dtype: int64

Check Data Types
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         474 non-null    object 
 1   latitude     473 non-null    float64
 2   longitude    473 non-null    float64
 3   designation  439 non-null    object 
 4   states       474 non-null    object 
 5   description  474 non-null    object 
 6   amenities    0 non-null      float64
 7   activities   464 non-null    object 
dtypes: float64(3), object(5)
memory usage: 29.8+ KB
None

Describe Dataframe
                                                     name    latitude  \
count                                                 474  473.000000   
unique                                     

- There are a few missing values - if we want to include those values on our map, these should be removed. These are latitude, longitude, designation, and activities
- Datatypes are all correct
- Amenities needs to be fixed (API)


- The instructions want a maximum of 9 geographical locations, so we will have to trim a subset of this data to use

### Visualization of All National Parks in the US

In [ ]:
parks = parks.dropna(subset=['latitude', 'longitude'])

# Example data (replace with your DataFrame)
data = {
    'name': parks['name'],
    'latitude': parks['latitude'],
    'longitude': parks['longitude']
}

df = pd.DataFrame(data)

fig = Figure(width=800, height=600)
m = Map(location=[39.8283, -98.5795], zoom_start=4)
fig.add_child(m)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name']
    ).add_to(m)

fig